<h1 align="center"><strong>Relazione di Open Data Management 2023-2024</strong></h1>
<h3 align="center"><strong><em>di Daniele Nicosia e Claudio Bellanti</em></strong></h3>
<h5 align="center"><em>Università degli Studi di Palermo - Facoltà di Informatica</em></h5>

***
***

### [`1. - Traccia`](#1-traccia)
### [`2. - Selezione dei dataset`](#2-selezione-dei-dataset)
- ##### [`2.1 - Raccolta`](#21-raccolta)
- ##### [`2.2 - Licenze`](#22-licenze)
### [`3. - Elaborazione dei dataset`](#3-elaborazione-dei-dataset)
- ##### [`3.1 - Pulizia e selezione dei dati rilevanti`](#31-pulizia-e-selezione-dei-dati-rilevanti)
- ##### [`3.2 - Arricchimento`](#32-arricchimento)
### [`4. - Finalizzazione dei dataset`](#4-finalizzazione-dei-dataset)
- ##### [`4.1 - Ontologia`](#41-ontologia)
- ##### [`4.2 - Conversione in RDF`](#42-conversione-in-rdf)
### [`5. - Visualizzazione dati`](#5-visualizzazione-dati)
- ##### [`5.1 - Creazione mappe con i GeoJson`](#51-creazione-mappe-con-i-geojson)

***
***

## **`1. - Traccia`**

Utilizzando il linguaggio di programmazione Python, per lo sviluppo del progetto si devono innanzitutto rispettare i seguenti passi:

- _Selezione dati_
- _Elaborazione dati (data cleaning, definizione struttura omogenea)_
- _Open Linked Data (creazione di uno strato semantico, ontologie, interlinking)_

L'obiettivo della suddetta relazione è lo sviluppo di ???

***
***

## **`2. - Selezione dei dataset`**

#### **`2.1 - Raccolta`**

Per lo sviluppo del progetto sono stati selezionati dei dataset i cui dati rappresentino gli eventi che si sono verificati nel Comune di Milano nel periodo 2014-2021. I dataset selezionati sono i seguenti:

- [**`Parco veicoli circolanti per classe di inquinamento 2007-2022`**](https://dati.comune.milano.it/dataset/ds530-emissioni-inquinanti-standard-eu-2007-avanti)
- [**`Ricoveri ordinari apparato respiratorio 2007-2021`**](https://dati.comune.milano.it/dataset/ds1053_ricoveri-ordinari-apparato-respiratorio)
- [**`Stazioni di monitoraggio inquinanti`**](https://dati.comune.milano.it/dataset/ds484_stazioni_di_monitoraggio_inquinanti_atmosferici_dellarpa_sit)
- [**`Ricostruzione della popolazione 2002-2019`**](https://demo.istat.it/app/?i=RIC&l=it)
  - Per questo dataset è stato necessario effettuare una personalizzazione tramite l'interfaccia di ISTAT. Il dataset risultante contiene la popolazione residente nel solo Comune di Milano dal 2002 al 2019.
- [**`Popolazione residente 2019-2023`**](https://demo.istat.it/app/?i=POS&l=it)
  - Per questo dataset è stato necessario effettuare una personalizzazione tramite l'interfaccia di ISTAT. Il dataset risultante contiene la popolazione residente nel solo Comune di Milano dal 2020 al 2022.
- [**`Standard di emissioni CO europei`**](https://en.wikipedia.org/wiki/European_emission_standards#Toxic_emission:_stages_and_legal_framework)
  - Per questo dataset è stato necessario effettuare uno scraping del sito. Il dataset risultante contiene gli standard di emissioni CO europei dal 1992 al 2021.

***


#### **`2.2 - Licenze`**

Le licenze dei dataset selezionati sono le seguenti:

- **`Parco veicoli circolanti per classe di inquinamento 2007-2022`**: Creative Commons Attribuzione 4.0 Internazionale ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/))
- **`Ricoveri ordinari apparato respiratorio 2007-2021`**: Creative Commons Attribuzione-Condividi allo stesso modo 3.0 Italia ([CC BY-SA 3.0 IT](https://creativecommons.org/licenses/by-sa/3.0/it/))
- **`Stazioni di monitoraggio inquinanti`**: Creative Commons Attribuzione 4.0 Internazionale ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/))
- **`Ricostruzione della popolazione 2002-2019`**: Creative Commons Attribuzione 3.0 ([CC BY 3.0](https://www.istat.it/it/note-legali))
- **`Popolazione residente 2019-2023`**: Creative Commons Attribuzione 3.0 ([CC BY 3.0](https://creativecommons.org/licenses/by/4.0/))
- **`Standard di emissioni CO europei`**: Creative Commons Attribuzione-ShareAlike 4.0 Internazionale ([CC BY-SA 4.0](https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_4.0_International_License))

***
***


## **`3. - Elaborazione dei dataset`**

#### **`3.1 - Pulizia e selezione dei dati rilevanti`**

Il primo passo effettuato è stato unire i diversi dataset della popolazione residente nel Comune di Milano, in un unico dataset.

Si è notato che i dataset puri non erano tutti adattati allo stesso modo, infatti, si sono riscontrate due problematiche:

- Encoding dei dataset differenti, a causa delle loro diversi origini
- Errori vari dovuti a `;` o `,` non correttamente inseriti

Inoltre, per il dataset delle emissioni è stato necessario effettuare uno scraping poichè non era disponibile in formato CSV. Per effettuare lo scraping è stato utilizzato il linguaggio di programmazione Python e la libreria `BeautifulSoup`. Lo script utilizzato è accessibile qui: [**`scraping.ipynb`**](dataset/emissioni/scraping.ipynb).

##### *`Preparazione ambiente di sviluppo`*

Prima di cominciare ad analizzare i dati dobbiamo importare le librerie necessarie al nostro scopo.



In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import re
import json
import plotly.graph_objects as go

##### *`Creazione dataset rilevazione della qualità dell'aria`*

In questo estratto di codice, ci siamo concentrati sull'elaborazione di un insieme di dati complesso riguardante la qualità dell'aria, raccolto nel corso di diversi anni. Questi dati provengono da molteplici stazioni che, è importante notare, hanno capacità di rilevamento diverse. Non tutte le stazioni sono equipaggiate per misurare ogni tipo di inquinante, il che introduce una certa variabilità nei dati che stiamo trattando.

Il primo passo significativo nel nostro processo è stato unire questi diversi file di dati, ognuno contenente frammenti del quadro generale della qualità dell'aria, in un unico, coerente DataFrame. Questo non solo ha semplificato le fasi successive dell'analisi, ma ha anche assicurato che ogni pezzo di informazione fosse contestualizzato all'interno del dataset più ampio.

Una volta consolidati i dati, abbiamo affrontato la questione della uniformità temporale, assicurandoci che ogni data fosse in un formato standard e ordinando poi il dataset in ordine cronologico. Questo ordine temporale è essenziale per qualsiasi analisi storica o trend che potrebbe emergere da questi dati.

Il problema successivo che abbiamo affrontato è stato quello dei valori mancanti, un'incidenza comune nei grandi set di dati, specialmente quando si tratta di rilevamenti ambientali su più anni. La strategia adottata qui è stata calcolare la mediana dei valori per ogni inquinante per ogni anno, una tecnica scelta per la sua resistenza agli outlier, ovvero quei valori che differiscono significativamente dalla norma e che potrebbero distorcere i risultati se non trattati correttamente.

Utilizzando le mediane annuali, abbiamo potuto imputare valori ragionevoli nei casi in cui i dati erano mancanti, mantenendo l'integrità generale dei dati senza concedere un peso eccessivo a misurazioni anomale o inaccurate. Questo passaggio è essenziale per mantenere la validità delle nostre conclusioni finali.

Dopo aver riempito questi vuoti, il nostro set di dati era quasi pronto per essere utilizzato in analisi future. Abbiamo rimosso alcune colonne superflue per rendere il dataset più snello e abbiamo gestito eventuali valori mancanti residui, assicurandoci che fossero chiaramente indicati.

***

In [ ]:
file_paths = glob.glob('./data/raw/rilevazione_qualità_aria/*.csv')

# Carica tutti i file CSV in un dizionario di DataFrame, specificando il separatore corretto
dfs = {fp: pd.read_csv(fp, sep=';') for fp in file_paths}

# Concatena tutti i DataFrame in un unico DataFrame
air_quality = pd.concat(dfs.values(), ignore_index=True)

# Converti la colonna 'data' in datetime se non lo è già
air_quality['data'] = pd.to_datetime(air_quality['data'])

# Ordina il DataFrame in base alla colonna 'data' in ordine crescente
air_quality.sort_values('data', ascending=True, inplace=True)

# Carica il file delle stazioni meteorologiche (adattalo al tuo percorso file)
stazioni = pd.read_csv('./data/raw/rilevazione_qualità_aria/stazioni_mereologiche.csv')

# Estraiamo gli inquinanti che ogni stazione può misurare e creiamo un dizionario
stazioni['inquinanti'] = stazioni['inquinanti'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
dict_stazioni_inquinanti = stazioni.set_index('id_amat')['inquinanti'].to_dict()

# Crea un nuovo DataFrame pivotato
air_quality_pivoted = pd.pivot_table(air_quality, values='valore', index=['stazione_id', 'data'], columns='inquinante').reset_index()
air_quality_pivoted['data'] = pd.to_datetime(air_quality_pivoted['data'])
air_quality_pivoted.sort_values('data', ascending=True, inplace=True)

# Estrai l'anno da ogni data
air_quality_pivoted['year'] = air_quality_pivoted['data'].dt.year

# Calcolo delle mediane annuali e riempimento dei valori NaN
annual_medians = {}
for stazione_id, inquinanti in dict_stazioni_inquinanti.items():
    for inquinante in inquinanti:
        if inquinante in air_quality_pivoted.columns:
            data_grouped = air_quality_pivoted[air_quality_pivoted['stazione_id'] == stazione_id].groupby('year')
            medians_by_year = data_grouped[inquinante].median()

            for year, median in medians_by_year.items():
                annual_medians[(stazione_id, inquinante, year)] = median

for key, median in annual_medians.items():
    stazione_id, inquinante, year = key
    mask = (air_quality_pivoted['stazione_id'] == stazione_id) & \
           (air_quality_pivoted['year'] == year) & \
           (air_quality_pivoted[inquinante].isna())
    
    air_quality_pivoted.loc[mask, inquinante] = median

# Rimozione della colonna 'year' dopo aver completato le operazioni di imputazione
air_quality_pivoted = air_quality_pivoted.drop(columns='year')
air_quality_pivoted = air_quality_pivoted.fillna("N.D.")

# Salvataggio del DataFrame aggiornato
air_quality_pivoted.to_csv('./data/processed/rilevazione_qualità_aria/air_quality_pivoted.csv', index=False, na_rep='NaN')

##### *`Creazione dataset GeoJson stazioni metereologiche`*

Il codice qui presentato si occupa essenzialmente di trasformare un elenco di stazioni meteorologiche, con varie informazioni associate, in un formato facilmente utilizzabile per applicazioni geospaziali. Inizialmente, vengono eliminate alcune informazioni non necessarie, concentrando l'attenzione su dati più pertinenti, come la posizione geografica di ogni stazione.

Una parte cruciale di questo processo riguarda la manipolazione delle coordinate geografiche. Dal momento che le posizioni sono state fornite come testo, il codice le converte in formato GeoJSON; uno standard specifico per i dati geografici.
Questo formato è particolarmente utile perché è ampiamente riconosciuto e utilizzato in molte applicazioni di mappatura, rendendo i dati facilmente accessibili e utilizzabili.
Alla fine di questa procedura avremo la possibilità, attraverso librerie e strumenti online, di ottenere una mappa con i nostri specifici "Point" e i relativi metadati.

In [ ]:
stazioni_metereologiche = pd.read_csv('./data/raw/rilevazione_qualità_aria/stazioni_mereologiche.csv')

stazioni_metereologiche.drop(columns=['inizio_operativita', 'fine_operativita','LONG_X_4326','LAT_Y_4326'], inplace=True)

# Estrai le coordinate dalla colonna "Location"
stazioni_metereologiche['coordinates'] = stazioni_metereologiche['Location'].str.strip('()').str.split(', ')
stazioni_metereologiche['latitude'] = stazioni_metereologiche['coordinates'].apply(lambda x: float(x[0]))
stazioni_metereologiche['longitude'] = stazioni_metereologiche['coordinates'].apply(lambda x: float(x[1]))

# Crea la struttura base GeoJSON
geojson = {
        "type": "FeatureCollection",
        "features": []
}

# Popola GeoJSON con le features
for index, row in stazioni_metereologiche.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": row.drop(['_id', 'Location', 'coordinates', 'latitude', 'longitude']).to_dict()
    }
    geojson["features"].append(feature)

# Salva il GeoJSON in un file
with open("./data/processed/rilevazione_qualità_aria/stazioni_metereologiche.geojson", 'w') as f:
    json.dump(geojson, f)

stazioni_metereologiche.to_csv('./data/processed/rilevazione_qualità_aria/stazioni_metereologiche.csv', index=False, na_rep='NaN')

##### *`Creazione dataset GeoJson aree verdi`*

Al fine di effettuare una analisi completa, abbiamo optato di inserire all'interno dell'analisi anche le aree verdi di Milano.
In questo caso abbiamo prelevato i dati forniti in CSV dal comune di Milano e ne abbiamo realizzato un Geo JSON.

In [ ]:
aree_verdi = pd.read_csv('./data/raw/rilevazione_qualità_aria/aree_verdi.csv')
if 'Location' in aree_verdi.columns:
    aree_verdi['coordinates'] = aree_verdi['Location'].str.strip('()').str.split(', ')
    aree_verdi['latitude'] = aree_verdi['coordinates'].apply(lambda x: float(x[0]))
    aree_verdi['longitude'] = aree_verdi['coordinates'].apply(lambda x: float(x[1]))
else:
    # Se non esiste una colonna 'Location', le coordinate potrebbero essere presenti come colonne separate
    # Assicurati che le colonne 'latitude' e 'longitude' esistano nel tuo file CSV
    aree_verdi['latitude'] = aree_verdi['latitude'].apply(float)  # Converti in float se già non lo sono
    aree_verdi['longitude'] = aree_verdi['longitude'].apply(float)  # Converti in float se già non lo sono

# Crea la struttura base GeoJSON
geojson_aree_verdi = {
    "type": "FeatureCollection",
    "features": []
}

# Popola GeoJSON con le features
for index, row in aree_verdi.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": row.drop(['Location', 'coordinates', 'latitude', 'longitude']).to_dict()  # escludi le colonne non necessarie
    }
    geojson_aree_verdi["features"].append(feature)

# Salva il GeoJSON in un file
output_file = "./data/processed/rilevazione_qualità_aria/aree_verdi.geojson"
with open(output_file, 'w') as f:
    json.dump(geojson_aree_verdi, f)

##### *`Creazione dataset emissioni veicoli`*
Il dataset in questione sulle emissioni dei veicoli basata sulla classificazione EURO è stato ottenuto effettuando web scraping.
Con esso siamo riusciti a prelevare da Wikipedia diverse tabelle dedicate a diversi tipi di veicolo, nonostante ciò il loro contenuto risultava particolarmente sporco e non completo.
Quindi, prima di poter utilizzare questi dati abbiamo dovuto effettuare un lavoro di pulizia manuale, in quanto automatizzare l'operazione risultava particolarmente onerosa e non la scelta ideale.
Per alcuni dataset è stato cambiato il delimitatore in quanto era utilizzato il punto e virgola, per una maggiore coerenza è stata sostituita con la virgola.
Sono stati inoltre standardizzati i valori della colonna "Tier" in quanto erano presenti in alcuni CSV i numeri romani piuttosto che quelli arabi.


In [ ]:
percorso_input = './data/raw/emissioni/'
percorso_output = './data/processed/emissioni/'

# Assicurati che le cartelle esistano, se no, creale
os.makedirs(percorso_output, exist_ok=True)

# Trova tutti i file CSV nel percorso di input
files = glob.glob(os.path.join(percorso_input, '*.csv'))

for file in files:
    file_name = os.path.basename(file)
    if file_name == 'standards_for_motor_cycles.csv':
        df = pd.read_csv(file, delimiter=',')
        if 'Standard' in df.columns:
            df.rename(columns={'Standard': 'Tier'}, inplace=True)
        df.to_csv(os.path.join(percorso_output, file_name), index=False, na_rep='NaN')
        continue  # Salta questo file

    # Leggi il CSV corrente
    df = pd.read_csv(file, delimiter=';')

    # Preparare il percorso del file di output
    nome_file = os.path.basename(file)  # Estrae il nome del file dal percorso
    file_di_output = os.path.join(percorso_output, nome_file)

    #Droppa le colonne che non ci servono
    if 'Date (type approval)' in df.columns:
        df.drop(columns=['Date (type approval)'], axis=0, inplace=True)
    #Cambiamo il nome delle colonne strane xd
    if 'Date (first registration)' in df.columns:
        df.rename(columns={'Date (first registration)': 'Date'}, inplace=True)
    if 'Standard' in df.columns:
        df.rename(columns={'Standard': 'Tier'}, inplace=True)

    # Elimino tutto ciò che non è un numero in tutte le righe della colonna Date
    df['Date'] = df['Date'].apply(lambda x: re.sub('[^0-9]', '', str(x)))

    df.to_csv(file_di_output, index=False, sep=',', na_rep='NaN')

##### *`Creazione dataset veicoli per classe di inquinamento`*

Il seguente dataset è stato ottenuto rimuovendo le colonne degli anni che non ci interessava analizzare. Inoltre le colonne sono state convertite in title-case e sono stati sostituiti eventuali valori NaN con 0.

In [ ]:
percorso_input = './data/raw/veicoli/categorie_euro_2007_2022.csv'
percorso_output = './data/processed/veicoli/categorie_euro_2014_2021.csv'

veicoli = pd.read_csv(percorso_input, sep=';')

veicoli = veicoli[veicoli['ANNO'].between(2014, 2021)]

veicoli = veicoli.reset_index(drop=True)

veicoli = veicoli.fillna(0)

veicoli.columns = veicoli.columns.str.title()

veicoli.to_csv(percorso_output, index=False)

##### *`Creazione dataset popolazione`*

Il dataset in questione è stato ottenuto, innanzitutto, rimuovendo le colonne degli anni che non ci interessava analizzare. Successivamente, è stato necessario unire dei dataset poichè sono stati cambiati i metadati e la loro disposizione negli anni successivi al 2019.

In [ ]:
percorso_input = './data/raw/popolazione/'
percorso_output = './data/processed/popolazione/'

popolazione = pd.read_csv(percorso_input + 'ricostruzione_popolazione_2002_2019.csv')

anni = np.arange(2014, 2021)
for col in popolazione.columns:
    try:
        year = int(col)
        if year not in anni:
            popolazione = popolazione.drop(columns=col)
    except ValueError:
        pass


file_list = os.listdir(percorso_input)
popolazione_files = [f for f in file_list if f.startswith('popolazione')]

prev_age = None
curr_row = None

for filename in popolazione_files:
    new_pop = pd.read_csv(os.path.join(percorso_input, filename))

    year = filename[-8:-4]
    popolazione[year] = 0

    for index, row in popolazione.iterrows():
        age = row['Età']
        if age != prev_age:
            curr_row = new_pop.loc[new_pop['Età'] == age].iloc[0]
            prev_age = age

        if popolazione.loc[index, 'Sesso'] == 'Maschi':
            popolazione.loc[index, year] = curr_row['Totale maschi']
        elif popolazione.loc[index, 'Sesso'] == 'Femmine':
            popolazione.loc[index, year] = curr_row['Totale femmine']
        else:
            popolazione.loc[index, year] = curr_row['Totale']

popolazione.to_csv(percorso_output + 'popolazione_2014_2021.csv', index=False)

#### **`3.2 - Arricchimento`**

Successivamente alla fase di pulizia e selezione, si è introdotta una fase di arricchimento dei dataset.

##### *`Arricchimento dataset emissioni`*

Poichè non siamo riusciti a trovare un dataset che contenesse il parco di veicoli in base ai kWh o alla cilindrata, abbiamo deciso di arricchire il dataset delle emissioni con le informazioni circa la media di questi valori, per ogni specifica classe di inquinamento. Abbiamo impostato quindi i valori in questo modo:

| Classe Euro   | kWh   | Media km percorsi |
|:-------------:|:-----:|:-----------------:|
| Euro 0        | 55    | 20                |
| Euro 1        | 60    | 20                |
| Euro 2        | 65    | 20                |
| Euro 3        | 70    | 20                |
| Euro 4        | 75    | 40                |
| Euro 5        | 80    | 50                |
| Euro 6        | 85    | 60                |

***
***

## **`4. - Finalizzazione dei dataset`**

#### **`4.1 - Ontologia`**

L'ontologia progettata in questione è di tipo `OWL` ed è stata realizzata, con l'ausilio del software `Protégé`, in modo da poter essere utilizzata per la rappresentazione di dati relativi alla qualità dell'aria in un determinato comune. Nell'ontologia sono rappresentate le seguenti principali entità o concetti:

- **`Comune`**: che rappresenta i comuni.
- **`Popolazione`**: che rappresenta la popolazione.
- **`AreaVerde`**: che rappresenta le aree verdi.
- **`StazioneMeteorologica`**: che rappresenta le stazioni meteorologiche.
- **`MisurazioneQualitaAria`**: che rappresenta le misurazioni della qualità dell'aria.
- **`Veicolo`**: che rappresenta i veicoli.
- **`ClasseInquinamento`**: che rappresenta le classi di inquinamento.

Sono state definite anche delle sottoclassi per categorizzare ulteriormente i veicoli (Motocicli, Autovetture, Autobus, Motrici, Industriali) e le classi di inquinamento (Euro0, Euro1, Euro2, Euro3, Euro4, Euro5, Euro6).

- **`haClasseInquinamento`**: Collega le entità per rappresentare l'appartenenza di un veicolo ad una classe di inquinamento specifica.
- **`haLimiteInquinante`**: Specifica il limite numerico associato a una classe di inquinamento.
- **`haRicoveri`**: Rappresenta il numero di ricoveri associati ad una popolazione.
- **`haPopolazione`**: Stabilisce il numero di abitanti per un dato comune.
- **`haAreaVerde`**: Indica la presenza di aree verdi all'interno di un comune.
- **`haStazioneMeteorologica`**: Rappresenta la presenza di stazioni meteorologiche in un comune.
- **`localizzataIn`**: Indica la posizione geografica di una stazione meteorologica o di un'area verde.
- **`monitoraInquinante`**: Specifica l'inquinante specifico misurato da una stazione meteorologica.
- **`haValoreInquinante`**: Indica il valore numerico effettivo di una misurazione della qualità dell'aria registrato da una stazione meteorologica.
- **`haQualitaAria`**: Collega un'area verde alle misurazioni della qualità dell'aria effettuate in quella zona.
- **`haDensitaPopolazione`**: Rappresenta la densità di popolazione in un'area verde.
- **`latitudine`** e **`longitudine`**: Specifica le coordinate geografiche (latitudine e longitudine) di stazioni meteorologiche e aree verdi.


L'ontologia descritta è stata realizzata nel formato `.ttl` ed è riportata in [**`questo file`**](ontologia.ttl).

***

In [ ]:
from urllib import parse 
from rdflib import Graph, Literal, URIRef, Namespace
from rdflib.namespace import RDF, OWL, RDFS
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [ ]:
g = Graph()

base_uri = "http://www.sanitasicilia.it/resource/"

sso = Namespace("http://www.semanticweb.org/aria-ontology-cb-dn/ontology/")
g.bind("sso", sso)

ssr = Namespace("http://www.semanticweb.org/aria-ontology-cb-dn/resource/")
g.bind("ssr", ssr)

def urify(uri, res):
    res = res.replace(" ","_").replace("\'","")
    return uri + parse.quote(res)

In [ ]:

# Carica il dataset sulla qualità dell'aria
air_quality_df = pd.read_csv('./data/processed/rilevazione_qualità_aria/air_quality_pivoted.csv')

# Prepara il grafo RDF
g = Graph()

# Prepara i namespace
wd = Namespace("http://www.wikidata.org/entity/")

# Prepara la query SPARQL
def get_wikidata_uris(pollutant_names):
    uris = []
    for pollutant_name in pollutant_names:
        encoded_pollutant_name = parse.quote_plus(pollutant_name)
        sparql_query = """
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?item WHERE {{
        ?item rdfs:label "{0}"@en.
        }}
        LIMIT 1
        """.format(pollutant_name)  # Non codificare qui, SPARQLWrapper gestisce già l'encoding
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery(sparql_query)
        sparql.setReturnFormat(JSON)
        try:
            results = sparql.query().convert()
            for result in results["results"]["bindings"]:
                uris.append(result["item"]["value"])
                break  # Break here ensures that only the first result is taken
        except Exception as e:
            print(f"An error occurred while querying {pollutant_name}: {e}")
            uris.append(None)  # Append None if there was an error or no result

    return uris

# Utilizza la nuova funzione per ottenere tutti gli URI
arr_inquinanti = ["benzene","carbon monoxide","nitrogen dioxide","ozone","PM10 (including PM 2.5)","PM10 (including PM 2.5)","sulfur dioxide"]
pollutant_uris = get_wikidata_uris(arr_inquinanti)

# Ora puoi iterare su pollutant_uris e creare le tue triple RDF
g = Graph()
for pollutant_name, pollutant_uri in zip(arr_inquinanti, pollutant_uris):
    print(pollutant_uri)
    if pollutant_uri:
        # Crea l'URIRef con l'URI trovato
        pollutant_uri_ref = URIRef(pollutant_uri)
        # Aggiungi le triple al grafo
        g.add((pollutant_uri_ref, RDF.type, OWL.Thing))
        g.add((pollutant_uri_ref, RDFS.label, Literal(pollutant_name)))

# Salva il grafo RDF
g.serialize(destination='./data/ontology/linked_air_quality.ttl', format='turtle')



#### **`4.2 - Conversione in RDF`**

???

***
***


## **`5. - Visualizzazione dati`**

#### **`5.1 - Creazione mappe con i GeoJson`**

Test

***